In [98]:
import pandas as pd

df = pd.read_csv("student_response.csv")

class_data = pd.read_csv("class_data.csv")

student_preferences = pd.read_csv("student_preferences.csv")


In [99]:
names = df['Name']
head_volunteer = df['head_volunteer']

df = df.drop(columns=['Name', 'attendance_likelihood', 'head_volunteer',
       'stay_all_day', 'event', 'specific_event', 'event.1',
       'specific_event.1', 'Which do you prefer?    =====================>',
       'Event grader', 'Proctor', 'Runner/Floater',
       'Would you be interested in being a photographer? \n\n0: No\n1: Yes'])
df.columns

Index(['Time_1', 'Time_2', 'Time_3', 'Time_4', 'Time_5', 'Time_6', 'Time_7',
       'Time_8'],
      dtype='object')

In [100]:
ind_to_event = {}
event_to_ind = {}

ind_to_name = {}
name_to_ind = {}

for i, name in enumerate(names):
    ind_to_name[i] = name
    name_to_ind[name] = i

for i, event in enumerate(class_data['Event']):
    ind_to_event[i] = event
    event_to_ind[event] = i

In [102]:
s_pref = {}

cols = student_preferences.columns[1:]

for i, row in student_preferences.iterrows():
    name = row['Name']
    name_ind = name_to_ind[name]
    class_inds = [index for index, value in enumerate(row[1:]) if value != 0]
    class_names = [event_to_ind[cols[ind]] for ind in class_inds]

    s_pref[name_ind] = class_names




In [ ]:
import gurobipy as grb

# Create a new model
model = grb.Model("example_lp")

model.ModelSense = grb.GRB.MINIMIZE

num_students = df.shape[0]
num_times = 8
num_classes = 32

# Dictionary to store decision variables

#vars[(i, j, k, 1)] = 1 if student i is a principal volunteer for class k at time j
#vars[(i, j, k, 0)] = 1 if student i is a non-principal volunteer for class k at time j

vars = {}

for i in range(num_students):
    for j in range(num_times):
        if not df.iloc[i, j]:
            # If volunteer not available, set variables to 0 and add constraints
            for k in range(num_classes):
                vars[(i, j, k, 0)] = model.addVar(name=f"x_{i}_{j}_{k}_np", vtype=grb.GRB.BINARY)
                vars[(i, j, k, 1)] = model.addVar(name=f"x_{i}_{j}_{k}_p", vtype=grb.GRB.BINARY)
                model.addConstr(vars[(i, j, k, 1)] == 0, name=f"no_volunteer_p_{i}_{j}_{k}")
                model.addConstr(vars[(i, j, k, 0)] == 0, name=f"no_volunteer_np_{i}_{j}_{k}")
            continue

        for k in range(num_classes):
            # Add decision variables
            vars[(i, j, k, 0)] = model.addVar(name=f"x_{i}_{j}_{k}_np", vtype=grb.GRB.BINARY)
            vars[(i, j, k, 1)] = model.addVar(name=f"x_{i}_{j}_{k}_p", vtype=grb.GRB.BINARY)

            # A person can either be a principal or non-principal volunteer
            model.addConstr(
                vars[(i, j, k, 0)] + vars[(i, j, k, 1)] <= 1, 
                name=f"princ_or_nonprinc_{i}_{j}_{k}"
            )

            # If not a head volunteer, they cannot be a principal volunteer
            if head_volunteer[i] != 1:
                model.addConstr(
                    vars[(i, j, k, 1)] == 0, 
                    name=f"not_head_volunteer_{i}_{j}_{k}"
                )

# A person can only volunteer for one class at a time
for i in range(num_students):
    for j in range(num_times):
        model.addConstr(
            sum(vars[(i, j, k, p)] for k in range(num_classes) for p in [0, 1]) <= 1,
            name=f"one_class_{i}_{j}"
        )

vars_to_minimize_slack = []

vars_to_minimize_constr = []

# Ensure the demand for each event is not exceeded
for j in range(num_times):
    for k in range(num_classes):
        vars_to_minimize_slack.append(model.addVar(name=f"demand_slack_{j}_{k}", lb=0))
        model.addConstr(
            sum(vars[(i, j, k, p)] for i in range(num_students) for p in [0, 1]) >= int(class_data['needed_volunteers'][k]) - vars_to_minimize_slack[-1],
            name=f"meet_demand_{j}_{k}"
        )

# Cost to meet demand as closely as possible
# minimize 

# minimize |vars[(i, j, k_1, p)] - vars[(i, j+1, k_2, p)]| for all i, j, p, k_1 neq k_2

vars_to_minimize = []

vars_to_minimize_constr = []

for i in range(num_students):
    for j in range(num_times-1):
        for k_1 in range(num_classes):
            for k_2 in range(num_classes):
                if k_1 == k_2:
                    continue
                for p in [0, 1]:
                    vars_to_minimize_constr.append(vars[(i, j, k_1, p)] - vars[(i, j+1, k_2, p)])
                    vars_to_minimize.append(model.addVar(name=f"minimize_{i}_{j}_{k_1}_{k_2}_{p}", lb=-grb.GRB.INFINITY))

                    model.addConstr(vars_to_minimize[-1] >= vars_to_minimize_constr[-1])
                    model.addConstr(vars_to_minimize[-1] >= -vars_to_minimize_constr[-1])                 


model.setObjectiveN(sum(vars_to_minimize_slack), 0)
model.setObjectiveN(sum(vars_to_minimize), 1)

# Add cost to assign students to thier preferred events

# Optimize the model
model.optimize()

# Check if an optimal solution was found
if model.status == grb.GRB.OPTIMAL:
    print("Optimal solution found.")
    print(f"Objective value: {model.objVal}")
    for key, var in vars.items():
        if var.x > 0:
            print(f"{var.varName} = {var.x}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1250U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2656240 rows, 1353856 columns and 7986432 nonzeros
Model fingerprint: 0x85159336
Variable types: 1305728 continuous, 48128 integer (48128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------